In [1]:
import firebase_admin
from firebase_admin import credentials, db
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# ================= CONFIGURASI =================
# 1. Lokasi & Waktu
LAT = -7.66  # Kebumen
LON = 109.65
# Format: YYYY-MM-DD
TARGET_DATE = "2025-11-29" 
START_TIME_WIB = "03:09:31"
END_TIME_WIB   = "07:46:31"
FORMAT_KEY = 'UNIX'  # <--- GANTI INI SESUAI DATABASE MU! (Saran: Coba UNIX dulu)

In [2]:
# 2. Firebase Config
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data" 

# ================= 1. SETUP KONEKSI =================
# Setup Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

# Setup Open-Meteo
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
client = openmeteo_requests.Client(session=retry_session)

In [6]:
def get_data_interpolated():
    url = "https://archive-api.open-meteo.com/v1/archive" 
    
    params = {
        "latitude": LAT, "longitude": LON,
        "start_date": TARGET_DATE, "end_date": TARGET_DATE,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "surface_pressure", "rain"],
        "timezone": "Asia/Jakarta"
    }
    
    print("🚀 Mengambil data Open-Meteo...")
    try:
        responses = client.weather_api(url, params=params)
        response = responses[0]
    except Exception as e:
        print(f"❌ Error API: {e}")
        return None

    hourly = response.Hourly()
    df = pd.DataFrame({
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(1).ValuesAsNumpy(),
        "dew": hourly.Variables(2).ValuesAsNumpy(),
        "pressure": hourly.Variables(3).ValuesAsNumpy(),
        "rain": hourly.Variables(4).ValuesAsNumpy()
    })
    
    # 1. Konversi ke WIB
    df = df.set_index('date')
    df.index = df.index.tz_convert('Asia/Jakarta')
    
    # 2. Resample ke Menit & Interpolasi
    df = df.resample('1T').asfreq()
    df = df.interpolate(method='time')
    
    # 3. Potong Waktu
    start = f"{TARGET_DATE} {START_TIME_WIB}"
    end   = f"{TARGET_DATE} {END_TIME_WIB}"
    df = df.loc[start:end]
    
    # 4. Cleaning
    df['rain'] = df['rain'].fillna(0) 
    df = df.dropna()
    
    return df

def upload_fix(df):
    ref = db.reference(DB_NODE_PATH)
    print(f"🔥 Memulai Upload ({len(df)} data) dengan Format UNIX (Int64)...")
    
    for index, row in df.iterrows():
        # --- FORMAT INT64 (UNIX TIMESTAMP) ---
        # Mengambil detik dari epoch (Contoh: 1764360540)
        # Kita cast jadi int dulu biar gak ada koma, terus jadi string buat Key Firebase
        unix_timestamp = int(index.timestamp())
        key = str(unix_timestamp) 
        
        data = {
            'temperature': round(float(row['temperature']), 2),
            'humidity': round(float(row['humidity']), 2),
            'dew': round(float(row['dew']), 2),
            'pressure': round(float(row['pressure']), 2),
            'rainrate': round(float(row['rain']), 2),
            'rainfall': round(float(row['rain']), 2),
            
            # Timestamp kita simpan juga dalam body sebagai referensi
            # Kalau frontend lu butuh string ISO, nyalain yang bawah.
            # Kalau frontend lu butuh angka (Unix) juga, pake yang atas.
            
            'timestamp': unix_timestamp, # Versi Angka (Int64)
            # 'timestamp_str': index.strftime('%Y-%m-%d %H:%M:%S'), # Versi Bacaan Manusia (Opsional)
        }
        
        try:
            ref.child(key).update(data)
            # Print log biar keliatan jalan
            if unix_timestamp % 600 == 0: # Print tiap 10 menit data
                print(f"   ✅ Key: {key} (Unix) -> Aman")
        except Exception as e:
            print(f"❌ Gagal pada {key}: {e}")
            
    print("✨ Selesai Upload! Semua key sekarang angka UNIX.")

if __name__ == "__main__":
    df_hasil = get_data_interpolated()
    
    if df_hasil is not None and not df_hasil.empty:
        print("\n" + "="*40)
        print("👀 PREVIEW (Pastikan Key-nya Angka)")
        print("="*40)
        
        # Ambil sampel pertama
        sampel_waktu = df_hasil.index[0]
        sampel_key = str(int(sampel_waktu.timestamp()))
        
        print(f"Waktu Asli : {sampel_waktu}")
        print(f"KEY (Unix) : {sampel_key}")
        
        x = input("\nGas upload? (y/n): ")
        if x.lower() == 'y':
            upload_fix(df_hasil)
        else:
            print("Batal.")
    else:
        print("Data kosong.")

🚀 Mengambil data Open-Meteo...

👀 PREVIEW (Pastikan Key-nya Angka)
Waktu Asli : 2025-11-29 03:10:00+07:00
KEY (Unix) : 1764360600
🔥 Memulai Upload (277 data) dengan Format UNIX (Int64)...
   ✅ Key: 1764360600 (Unix) -> Aman
   ✅ Key: 1764361200 (Unix) -> Aman
   ✅ Key: 1764361800 (Unix) -> Aman
   ✅ Key: 1764362400 (Unix) -> Aman
   ✅ Key: 1764363000 (Unix) -> Aman
   ✅ Key: 1764363600 (Unix) -> Aman
   ✅ Key: 1764364200 (Unix) -> Aman
   ✅ Key: 1764364800 (Unix) -> Aman
   ✅ Key: 1764365400 (Unix) -> Aman
   ✅ Key: 1764366000 (Unix) -> Aman
   ✅ Key: 1764366600 (Unix) -> Aman
   ✅ Key: 1764367200 (Unix) -> Aman
   ✅ Key: 1764367800 (Unix) -> Aman
   ✅ Key: 1764368400 (Unix) -> Aman
   ✅ Key: 1764369000 (Unix) -> Aman
   ✅ Key: 1764369600 (Unix) -> Aman
   ✅ Key: 1764370200 (Unix) -> Aman
   ✅ Key: 1764370800 (Unix) -> Aman
   ✅ Key: 1764371400 (Unix) -> Aman
   ✅ Key: 1764372000 (Unix) -> Aman
   ✅ Key: 1764372600 (Unix) -> Aman
   ✅ Key: 1764373200 (Unix) -> Aman
   ✅ Key: 1764373800

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>